In [1]:
from time import time
import unittest
import numpy as np
from sklearn.metrics import silhouette_score

import os
os.environ["CUDA_VISIBLE_DEVICES"] = '1' 

from keras.datasets import mnist
import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
# config.gpu_options.per_process_gpu_memory_fraction = 0.48
set_session(tf.Session(config=config))

import sys  
sys.path.append('../')
from util.util import *
from util import importNotebook
from trial.autoencoder import VAE, ConvVAE

Using TensorFlow backend.


importing Jupyter notebook from ../trial/autoencoder.ipynb


In [54]:
class TestVAE(unittest.TestCase):
    
    numEpochs = 10
    sizeBatch = 128
    sizeKernel = 3
    layerDense = [16, 2]
    stdEps = 1.0 
    ratRecon = 0.998
    factNoise = 0
    nameOptim = 'adam'
    modelPath = '../model/temp/'
    patience = 3
    
    (xTrain, yTrain), (xTest, yTest) = mnist.load_data()
    xTrain = xTrain.astype('float32') / 255.
    xTest = xTest.astype('float32') / 255.
    numTrain = len(xTrain)
    numTest = len(xTest)
    numClass = 10
    sizeDigit = xTrain.shape[1:]

    dimInput = np.prod(xTrain.shape[1:])
    xTrain = xTrain.reshape((numTrain, dimInput))
    xTest = xTest.reshape((numTest, dimInput))
    print(xTrain.shape)
    print(xTest.shape)
    
    def test_init(self):
        print(self.dimInput)
        vae = VAE(784, layerDense=self.layerDense)
        vae.encoder.summary()
        vae.decoder.summary()
        vae.autoencoder.summary()
        self.vae = vae

    def test_train(self):
        history, timeTrain = self.vae.train(self.xTrain, self.xTest, 
                                       numEpochs=self.numEpochs,
                                       sizeBatch=self.sizeBatch,
                                       tempPathBest=self.modelPath)
    
    def exampUsage(self):
        xTrain = self.xTrain
        xTest = self.xTest
        
        # Initialization
        vae = VAE(np.array([784]))
#         vae = VAE([784], layerDense=[64, 2], actDense='relu', ratRecon=0.998)

        # models summary
        vae.encoder.summary()
        vae.decoder.summary()
        vae.autoencoder.summary()
    
        # training
        history, timeTrain = vae.train(xTrain, xTest)
#         history, timeTrain = vae.train(xTrain, xTest, 
#                                        numEpochs=10,
#                                        sizeBatch=32
#                                        tempPathBest='./temp')

        # prediction
        encodeTest = vae.encoder.predict(xTest)
        decodeTest = vae.decoder.predict(encodeTest)
#         decodeTest = vae.autoencoder.predict(xTest)

    def testPrediction(self):
        xTrain = self.xTrain
        xTest = self.xTest
        yTest = self.yTest
        sizeDigit = self.sizeDigit
        vae = self.vae
        encoder = vae.encoder
        decoder = vae.decoder
        
        encodeTest = encoder.predict(xTest)
        decodeTest = decoder.predict(encodeTest)
        meanEncTest = np.mean(encodeTest, axis=0)
        stdEncTest = np.std(encodeTest, axis=0)
        
        plotCompDecode(xTest, decodeTest, sizeDigit=sizeDigit)
        
        xlim = (meanEncTest[0] - 4*stdEncTest[0], meanEncTest[0] + 4*stdEncTest[0])
        ylim = (meanEncTest[1] - 4*stdEncTest[1], meanEncTest[1] + 4*stdEncTest[1])

        plotScatterEncode(encodeTest, yTest, xlim, ylim, numShow=10000)
        scoreSilh = silhouette_score(encodeTest, yTest)  
        
        # display a 2D manifold of the digits
        plotScatterDecode(decoder, sizeDigit, xlim, ylim)

784
(60000, 784)
(10000, 784)


In [65]:
class TestConvVAE(unittest.TestCase):
    
    numEpochs = 10
    sizeBatch = 128
    sizeKernel = 3
    layerDense = [16, 2]
    layerConv = [4, 16]
    stdEps = 1.0 
    ratRecon = 0.998
    factNoise = 0
    nameOptim = 'adam'
    modelPath = '../model/temp/'
    patience = 3
    
    (xTrain, yTrain), (xTest, yTest) = mnist.load_data()
    xTrain = xTrain.astype('float32') / 255.
    xTest = xTest.astype('float32') / 255.
    numTrain = len(xTrain)
    numTest = len(xTest)
    numClass = 10
    sizeDigit = xTrain.shape[1:]
    dimInput = [*xTrain.shape[1:], 1]
    xTrain = xTrain.reshape((numTrain, *dimInput))
    xTest = xTest.reshape((numTest, *dimInput))
    print(xTrain.shape)
    print(xTest.shape)
    
    def test_init(self):
        convVAE = ConvVAE(self.dimInput, layerDense=self.layerDense, layerConv=self.layerConv)
        convVAE.encoder.summary()
        convVAE.decoder.summary()
        convVAE.autoencoder.summary()
        self.convVAE = convVAE

    def test_train(self):
        history, timeTrain = self.convVAE.train(self.xTrain, self.xTest, 
                                                numEpochs=self.numEpochs,
                                                sizeBatch=self.sizeBatch,
                                                tempPathBest=self.modelPath)
        
    def testPrediction(self):
        xTrain = self.xTrain
        xTest = self.xTest
        yTest = self.yTest
        sizeDigit = self.sizeDigit
        convVAE = self.convVAE
        encoder = convVAE.encoder
        decoder = convVAE.decoder
        
        encodeTest = encoder.predict(xTest)
        decodeTest = decoder.predict(encodeTest)
        meanEncTest = np.mean(encodeTest, axis=0)
        stdEncTest = np.std(encodeTest, axis=0)
        
        plotCompDecode(xTest, decodeTest, sizeDigit=sizeDigit)
        
        xlim = (meanEncTest[0] - 4*stdEncTest[0], meanEncTest[0] + 4*stdEncTest[0])
        ylim = (meanEncTest[1] - 4*stdEncTest[1], meanEncTest[1] + 4*stdEncTest[1])

        plotScatterEncode(encodeTest, yTest, xlim, ylim, numShow=10000)
        scoreSilh = silhouette_score(encodeTest, yTest)  
        
        # display a 2D manifold of the digits
        plotScatterDecode(decoder, sizeDigit, xlim, ylim)

(60000, 28, 28, 1)
(10000, 28, 28, 1)


In [66]:
if __name__ == '__main__':
#     testVAE = TestVAE()
#     testVAE.test_init()
#     testVAE.test_train()
#     testVAE.testPrediction()
    
    testConv = TestConvVAE()
    testConv.test_init()
    testConv.test_train()
    testConv.testPrediction()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_18 (InputLayer)           (None, 28, 28, 1)    0                                            
__________________________________________________________________________________________________
conv2d_13 (Conv2D)              (None, 14, 14, 4)    40          input_18[0][0]                   
__________________________________________________________________________________________________
conv2d_14 (Conv2D)              (None, 7, 7, 16)     592         conv2d_13[0][0]                  
__________________________________________________________________________________________________
flatten_7 (Flatten)             (None, 784)          0           conv2d_14[0][0]                  
__________________________________________________________________________________________________
dense_68 (

KeyboardInterrupt: 